# Finding Locations to Open a Bar in Amsterdam

## 1. Import libraries we needed

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.



## 2.  Fetch and Consolidate data from different sources

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Amsterdam').text
soup = BeautifulSoup(url,'html.parser')


In [3]:
#print(soup.prettify())

#### create a list to store neighborhood data and append all contents in the li tag to the list

In [4]:

neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

#### convert list into a dataframe but drop the first item which shows : Neighborhoods of Amsterdam

In [5]:

AMS_df = pd.DataFrame({"Neighborhood": neighborhoodList[1:]})
AMS_df.head()
AMS_df.shape

(105, 1)

In [6]:
# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Amsterdam, Netherlands'.format(postal_code))
#   lat_lng_coords = g.latlng
# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]


# define a function to get coordinates
# Remember to install gecoder by: pip install geocoder
# And import geocoder: import geocoder

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Amsterdam, Netherlands'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [7]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in AMS_df["Neighborhood"].tolist() ]
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_coords.to_csv("df_coords.csv", index=False)


In [8]:
# merge the coordinates into the AMS_df dataframe
AMS_df['Latitude'] = df_coords['Latitude']
AMS_df['Longitude'] = df_coords['Longitude']
AMS_df.head(15)

,Neighborhood,Latitude,Longitude
0,Admiralenbuurt,52.372728,4.856362
1,Amsteldorp,52.360420,4.905250
2,Amsterdam Oud-West,52.365390,4.870220
3,Amsterdam Oud-Zuid,52.352350,4.877880
4,Amsterdam Science Park,52.354320,4.958030
5,Apollobuurt,52.350300,4.868196
6,Betondorp,52.423398,4.833391
7,Bijlmermeer,52.309893,4.974430
8,Binnenstad (Amsterdam),52.369930,4.907880
9,Bos en Lommer,52.379190,4.851740


In [9]:
print(AMS_df.shape)

(105, 3)


In [10]:
AMS_df.to_csv("AMS_df.csv", index=False)

In [11]:
# get the coordinates
address = 'Amsterdam, Netherlands'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Amsterdam, Netherlands {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Amsterdam, Netherlands 52.3745403, 4.89797550561798.


####  create map of Amsterdam using latitude and longitude values

In [12]:

map_AMS = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(AMS_df['Latitude'], AMS_df['Longitude'], AMS_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_AMS)  
    
map_AMS

In [13]:
# save the map as HTML file
map_AMS.save('map_AMS.html')

### Load data via Foursquare Developer API

In [2]:
# define Foursquare Credentials and Version
CLIENT_ID = 'xxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxxx
CLIENT_SECRET:xxxxx


#### Pull data from Foursquare API

In [19]:
LIMIT = 300
radius = 3000


venues = []

for lat, long, neighborhood in zip(AMS_df['Latitude'], AMS_df['Longitude'], AMS_df['Neighborhood']): 
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT) 
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        

In [63]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)
# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)



(10021, 7)


In [64]:
venues_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Admiralenbuurt,52.372728,4.856362,Radijs,52.371049,4.856756,Bistro
1,Admiralenbuurt,52.372728,4.856362,Café Cook,52.371208,4.852792,Pub
2,Admiralenbuurt,52.372728,4.856362,T's,52.370527,4.853809,Tea Room
3,Admiralenbuurt,52.372728,4.856362,White Label Coffee,52.370346,4.851584,Coffee Shop
4,Admiralenbuurt,52.372728,4.856362,Chassé Dance Studios,52.370058,4.861566,Dance Studio


In [22]:
venues_df.to_csv("venues_df.csv", index=False)

In [66]:
venues_df.groupby(["Neighborhood"]).count()


,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Admiralenbuurt,100,100,100,100,100,100
Amsteldorp,100,100,100,100,100,100
Amsterdam Oud-West,100,100,100,100,100,100
Amsterdam Oud-Zuid,100,100,100,100,100,100
Amsterdam Science Park,100,100,100,100,100,100
Apollobuurt,100,100,100,100,100,100
Betondorp,95,95,95,95,95,95
Bijlmermeer,100,100,100,100,100,100
Binnenstad (Amsterdam),100,100,100,100,100,100


In [75]:
df_rank= venues_df.VenueCategory.value_counts()

df_Rank1 = pd.DataFrame({"Count": df_rank[0:]})
df_Rank1.head(10)


,Count
Hotel,667
Coffee Shop,454
Restaurant,449
Café,375
Bar,373
French Restaurant,292
Park,223
Bakery,203
Breakfast Spot,169
Bookstore,153


In [30]:

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 256 uniques categories.


In [31]:
venues_df['VenueCategory'].unique()[:50]

array(['Bistro', 'Pub', 'Tea Room', 'Coffee Shop', 'Dance Studio',
       'Latin American Restaurant', 'Falafel Restaurant', 'Café',
       'Pizza Place', 'Bar', 'Restaurant', 'Fried Chicken Joint',
       'Italian Restaurant', 'Burger Joint', 'Mexican Restaurant',
       'Wine Bar', 'Hostel', 'Yoga Studio', 'Indonesian Restaurant',
       'Food Court', 'Hotel', 'Moroccan Restaurant', 'Movie Theater',
       'Grocery Store', 'Cocktail Bar', 'Bookstore', 'Park',
       'Garden Center', 'Korean Restaurant', 'Dutch Restaurant',
       'Caribbean Restaurant', 'Furniture / Home Store',
       'Marijuana Dispensary', 'Clothing Store', 'Chocolate Shop',
       'Music Venue', 'Cycle Studio', 'Diner', 'BBQ Joint', 'Cheese Shop',
       'Seafood Restaurant', 'Deli / Bodega', 'Ice Cream Shop', 'Brewery',
       'Gift Shop', 'French Restaurant', 'Flea Market',
       'General Entertainment', 'Hotel Bar', 'Bakery'], dtype=object)

In [32]:
# check if the results
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [33]:
# one hot encoding
AMS_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
AMS_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [AMS_onehot.columns[-1]] + list(AMS_onehot.columns[:-1])
AMS_onehot = AMS_onehot[fixed_columns]

print(AMS_onehot.shape)
AMS_onehot.head()

(10021, 257)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Field,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lighthouse,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Admiralenbuurt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
AMS_grouped = AMS_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(AMS_grouped.shape)
AMS_grouped

(105, 257)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Field,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lighthouse,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Admiralenbuurt,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.01,0.000000,0.000000,0.010000,0.080000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.000000,0.020000,0.00,0.00,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.00,0.0

#### Let's confirm the new size

In [35]:
AMS_grouped.shape

(105, 257)

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = AMS_grouped['Neighborhoods']

for ind in np.arange(AMS_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(AMS_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Admiralenbuurt,Bar,Coffee Shop,Hotel,Yoga Studio,Pizza Place,Café,Park,Restaurant,Caribbean Restaurant,Cheese Shop
1,Amsteldorp,Hotel,Coffee Shop,Breakfast Spot,Restaurant,Bakery,Yoga Studio,Café,Seafood Restaurant,Bar,Canal
2,Amsterdam Oud-West,Hotel,Art Museum,Coffee Shop,Café,Bookstore,Music Venue,Bar,Yoga Studio,Burger Joint,Italian Restaurant
3,Amsterdam Oud-Zuid,Art Museum,Hotel,Café,Breakfast Spot,Coffee Shop,Japanese Restaurant,Seafood Restaurant,Music Venue,Restaurant,Bar
4,Amsterdam Science Park,Bar,Restaurant,Coffee Shop,French Restaurant,Hotel,Ice Cream Shop,Italian Restaurant,Café,Park,Vegetarian / Vegan Restaurant
5,Apollobuurt,Café,Art Museum,Hotel,Breakfast Spot,Japanese Restaurant,Coffee Shop,Seafood Restaurant,Music Venue,Deli / Bodega,Yoga Studio
6,Betondorp,Restaurant,Hotel,Fast Food Restaurant,Bar,Furniture / Home Store,Supermarket,Gym,Nightclub,Coffee Shop,Clothing Store
7,Bijlmermeer,Hotel,Supermarket,Restaurant,Coffee Shop,Park,Café,Concert Hall,Chinese Restaurant,Plaza,Sporting Goods Shop
8,Binnenstad (Amsterdam),Hotel,French Restaurant,Coffee Shop,Bookstore,Sandwich Place,Bakery,Bar,Restaurant,Breakfast Spot,Burger Joint
9,Bos en Lommer,Coffee Shop,Bar,Hotel,Café,Italian Restaurant,Pizza Place,Yoga Studio,Restaurant,Bistro,Park


#### Let's create a data frame of Bars in Amsterdam

In [38]:
AMS_BAR = AMS_grouped[["Neighborhoods","Bar"]]
AMS_BAR.head()

,Neighborhoods,Bar
0,Admiralenbuurt,0.08
1,Amsteldorp,0.02
2,Amsterdam Oud-West,0.03
3,Amsterdam Oud-Zuid,0.03
4,Amsterdam Science Park,0.08


#### Cluster Neighborhoods

In [39]:
# set number of clusters
amsclusters = 5

ams_clustering = AMS_BAR.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=amsclusters, random_state=0).fit(ams_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 2, 2, 3, 2, 2, 1, 2, 3], dtype=int32)

In [40]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ams_merged = AMS_BAR.copy()

# add clustering labels
ams_merged["Cluster Labels"] = kmeans.labels_

In [41]:
ams_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
ams_merged.head()

,Neighborhood,Bar,Cluster Labels
0,Admiralenbuurt,0.08,3
1,Amsteldorp,0.02,1
2,Amsterdam Oud-West,0.03,2
3,Amsterdam Oud-Zuid,0.03,2
4,Amsterdam Science Park,0.08,3


In [42]:
# merge ams_grouped with toronto_data to add latitude/longitude for each neighborhood
ams_merged = ams_merged.join(AMS_df.set_index("Neighborhood"), on="Neighborhood")


In [43]:
print(ams_merged.shape)
ams_merged.head(20)

(105, 5)


,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
0,Admiralenbuurt,0.080000,3,52.372728,4.856362
1,Amsteldorp,0.020000,1,52.360420,4.905250
2,Amsterdam Oud-West,0.030000,2,52.365390,4.870220
3,Amsterdam Oud-Zuid,0.030000,2,52.352350,4.877880
4,Amsterdam Science Park,0.080000,3,52.354320,4.958030
5,Apollobuurt,0.030000,2,52.350300,4.868196
6,Betondorp,0.031579,2,52.423398,4.833391
7,Bijlmermeer,0.010000,1,52.309893,4.974430
8,Binnenstad (Amsterdam),0.030000,2,52.369930,4.907880
9,Bos en Lommer,0.070000,3,52.379190,4.851740


In [44]:
# sort the results by Cluster Labels
print(ams_merged.shape)
ams_merged.sort_values(["Cluster Labels"], inplace=True)
ams_merged

(105, 5)


,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
52,NDSM,0.060000,0,52.400129,4.897116
25,Eastern Docklands,0.060000,0,52.387066,4.905946
37,Indische Buurt,0.060000,0,52.402390,4.649700
40,Jordaan,0.060000,0,52.376870,4.879270
44,Kinkerbuurt,0.050000,0,52.364687,4.859795
19,Dapperbuurt,0.060000,0,52.362763,4.927371
46,Landelijk Noord,0.050000,0,52.400720,4.935270
16,Chassébuurt,0.060000,0,52.369180,4.861475
103,Zeeheldenbuurt,0.060000,0,53.213490,6.550940
59,Omval,0.050000,0,52.344062,4.914569


### CLUSTER MAP

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(amsclusters)
ys = [i+x+(i*x)**2 for i in range(amsclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ams_merged['Latitude'], ams_merged['Longitude'], ams_merged['Neighborhood'], ams_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [46]:
# save the map as HTML file
map_clusters.save('mapAms_clusters.html')

#### Examine Clusters

In [47]:
ams_merged.loc[ams_merged['Cluster Labels'] == 0]


,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
52,NDSM,0.060000,0,52.400129,4.897116
25,Eastern Docklands,0.060000,0,52.387066,4.905946
37,Indische Buurt,0.060000,0,52.402390,4.649700
40,Jordaan,0.060000,0,52.376870,4.879270
44,Kinkerbuurt,0.050000,0,52.364687,4.859795
19,Dapperbuurt,0.060000,0,52.362763,4.927371
46,Landelijk Noord,0.050000,0,52.400720,4.935270
16,Chassébuurt,0.060000,0,52.369180,4.861475
103,Zeeheldenbuurt,0.060000,0,53.213490,6.550940
59,Omval,0.050000,0,52.344062,4.914569


In [48]:

ams_merged.loc[ams_merged['Cluster Labels'] == 1]


,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
29,Gaasperdam,0.010204,1,52.296800,4.978120
54,Nieuw Sloten,0.010000,1,52.345610,4.811920
69,Oude Pijp,0.020000,1,52.356250,4.890570
47,Landlust,0.020000,1,52.346566,4.973029
74,Prinses Irenebuurt,0.020000,1,52.341760,4.875243
85,Stadionbuurt,0.020000,1,52.349516,4.884687
45,Kolenkit District,0.020000,1,52.377660,4.842810
90,Tuindorp Nieuwendam,0.014925,1,52.387891,4.968316
36,IJburg,0.020000,1,52.365743,4.975558
56,Nieuwendam,0.014925,1,52.387891,4.968316


In [49]:
ams_merged.loc[ams_merged['Cluster Labels'] == 2]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
57,Nieuwendammerdijk en Buiksloterdijk,0.040000,2,52.386746,4.947707
58,Olympisch Kwartier,0.030000,2,52.369930,4.907880
88,Transvaalbuurt (Amsterdam),0.040000,2,52.351276,4.918909
87,Teleport (Amsterdam),0.030000,2,52.387700,4.841870
2,Amsterdam Oud-West,0.030000,2,52.365390,4.870220
86,Steigereiland,0.030000,2,52.369930,4.907880
63,Oostoever,0.040000,2,52.370000,4.830460
11,Buikslotermeer,0.040000,2,52.396390,4.935390
65,Oostzanerwerf,0.031746,2,52.425350,4.885635
61,Oosterdokseiland,0.030000,2,52.369930,4.907880


In [50]:
ams_merged.loc[ams_merged['Cluster Labels'] == 3]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
4,Amsterdam Science Park,0.080000,3,52.354320,4.958030
83,Spaarndammerbuurt,0.070000,3,52.388683,4.879929
84,Staatsliedenbuurt (Amsterdam),0.070000,3,52.380050,4.871490
0,Admiralenbuurt,0.080000,3,52.372728,4.856362
9,Bos en Lommer,0.070000,3,52.379190,4.851740
62,Oosterparkbuurt (Amsterdam),0.070000,3,52.359470,4.922980
60,Oostelijke Eilanden,0.070000,3,52.368570,4.928600
98,Westelijke Eilanden (Amsterdam),0.070000,3,52.368570,4.928600
51,Museumkwartier (Amsterdam),0.070000,3,52.362290,4.923910
50,Molenwijk (Amsterdam),0.069767,3,52.421005,4.890154


In [51]:
ams_merged.loc[ams_merged['Cluster Labels'] == 4]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
97,Westelijk Havengebied,0.0,4,52.405780,4.784290
26,Eendracht (Amsterdam),0.0,4,52.381560,4.784360
75,Ransdorp,0.0,4,52.393060,4.994260
67,Oud Osdorp,0.0,4,52.356360,4.790600
12,Buitenveldert,0.0,4,52.330930,4.875180
55,Nieuwe Pijp,0.0,4,52.362002,4.799183
49,Middelveldsche Akerpolder,0.0,4,52.341765,4.796769
20,De Aker,0.0,4,52.352804,4.776768
31,Gouden Reael,0.0,4,52.326072,4.865346
93,Venserpolder,0.0,4,52.326730,4.945590


In [52]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)
# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head(20)

(10021, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Admiralenbuurt,52.372728,4.856362,Radijs,52.371049,4.856756,Bistro
1,Admiralenbuurt,52.372728,4.856362,Café Cook,52.371208,4.852792,Pub
2,Admiralenbuurt,52.372728,4.856362,T's,52.370527,4.853809,Tea Room
3,Admiralenbuurt,52.372728,4.856362,White Label Coffee,52.370346,4.851584,Coffee Shop
4,Admiralenbuurt,52.372728,4.856362,Chassé Dance Studios,52.370058,4.861566,Dance Studio
5,Admiralenbuurt,52.372728,4.856362,Ram's Roti West,52.374865,4.859968,Latin American Restaurant
6,Admiralenbuurt,52.372728,4.856362,Kauffmann - Falafel & Beer,52.371470,4.862502,Falafel Restaurant
7,Admiralenbuurt,52.372728,4.856362,Local Hero,52.374947,4.860698,Café
8,Admiralenbuurt,52.372728,4.856362,Eetwinkel Buurman & Buurman,52.375401,4.848373,Pizza Place
9,Admiralenbuurt,52.372728,4.856362,Bar Spek,52.370078,4.864525,Bar


In [53]:
is_bar = venues_df['VenueCategory'] =="Bar"

In [54]:
bar_df = venues_df[is_bar]

bar_df.shape

(373, 7)

In [55]:
bar_df.head(15)

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
9,Admiralenbuurt,52.372728,4.856362,Bar Spek,52.370078,4.864525,Bar
27,Admiralenbuurt,52.372728,4.856362,Gin & Tonic Bar,52.367101,4.868080,Bar
47,Admiralenbuurt,52.372728,4.856362,l'Affiche,52.366572,4.874748,Bar
49,Admiralenbuurt,52.372728,4.856362,Maloe Melo,52.369981,4.877567,Bar
55,Admiralenbuurt,52.372728,4.856362,Gollem's Proeflokaal,52.362206,4.873080,Bar
65,Admiralenbuurt,52.372728,4.856362,Café Chris,52.374224,4.881570,Bar
75,Admiralenbuurt,52.372728,4.856362,Belgisch Bierproeflokaal De Zotte,52.366195,4.880800,Bar
85,Admiralenbuurt,52.372728,4.856362,Café 't Smalle,52.376700,4.884243,Bar
119,Amsteldorp,52.360420,4.905250,Bar Bukowski,52.358088,4.917527,Bar
133,Amsteldorp,52.360420,4.905250,Café Brecht,52.359609,4.890243,Bar


In [56]:
bar_rank= bar_df.Neighborhood.value_counts()

In [77]:
bar_dfRank = pd.DataFrame(bar_rank)
bar_dfRank1 = pd.DataFrame({"Count": bar_rank[0:]})
bar_dfRank1.head(15)

,Count
Houthaven,8
Amsterdam Science Park,8
Watergraafsmeer,8
Admiralenbuurt,8
Westelijke Eilanden (Amsterdam),7
Bos en Lommer,7
Staatsliedenbuurt (Amsterdam),7
Spaarndammerbuurt,7
Museumkwartier (Amsterdam),7
Oosterparkbuurt (Amsterdam),7


In [78]:
bar_dfRank1.tail(10)

,Count
Stadionbuurt,2
Bijlmermeer,1
Nieuw Sloten,1
Zuidas,1
Tuindorp Nieuwendam,1
Ruigoord,1
Nieuwendam,1
Burgwallen Nieuwe Zijde,1
Gaasperdam,1
Bullewijk,1
